# AMPIS tutorial
<img src="https://github.com/rccohn/AMPIS/blob/master/.github/particles_screenshot.png?raw=true" width=750>


Welcome to ampis! This is the official colab tutorial of ampis. Here, we will go through some basics usage of detectron2, including the following:
* Train a model on a dataset of powder images
* Run model inference

You can make a copy of this tutorial by "File --> Save a copy in Drive."

Copyright (c) 2020 Ryan Cohn and Elizabeth Holm. All rights reserved. <br />
Licensed under the MIT License (see LICENSE for details) <br />
Tutorial written by [Ryan Cohn](https://github.com/rccohn).


# Install detectron2 and AMPIS
These cells should be run ONCE to install the required libraries. After the initial installation of detectron2 and AMPIS, the runtime must be restarted (Runtime --> Restart runtime). Otherwise the libraries will not be able to be imported.

The last line, exit(), will end the current runtime. You may get a warning from colab that the session 'crashed' for an unknown reason. This is likely that reason. Just proceed as usual.

In [ ]:
try: # if libraries are not installed, then install them. Otherwise, proceed normally.
  import ampis
  import detectron2

except ImportError:
  # detectron2  installation
  !pip install pyyaml==5.1
  # workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
  !pip uninstall -y torch torchvision torchtext
  !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

  # install detectron2 that matches pytorch 1.8
  # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
  !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
  # exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

  # get AMPIS
  # note we clone the repo instead of simply installing from git
  # so that the example data is easier to work with
  !git clone https://github.com/rccohn/AMPIS.git

  # install ampis
  # note after installation you must restart the runtime
  # otherwise you will run into "No module named 'ampis'"
  !pip install -e AMPIS
  exit(0) # restarts runtime


## Verify installation
If everything is installed correctly, this cell should run without errors

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.8")   # please manually install torch 1.8 if Colab changes its default version

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# ampis
import ampis

# Loading data and model training
This example will take you through the process of training a model to segment powder particles and visualizing the predictions.

## Module imports

In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import pickle
import sys

## detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
)
from detectron2.engine import DefaultTrainer, DefaultPredictor

from ampis import data_utils, visualize

%matplotlib inline

## Labeling Data

The recommended tool for labeling is the [VGG Image Annotator](http://www.robots.ox.ac.uk/~vgg/software/via/) To save you the trouble of having to annotate data yourself, existing labels are available under ampis/examples/powder/data/via_2.0.8. For labeling new datasets we follow the same process described in the 'balloon example' [here](https://engineering.matterport.com/splash-of-color-instance-segmentation-with-mask-r-cnn-and-tensorflow-7c761e238b46).





## Loading Data
We need to specify the path to the VIA annotation files.
The paths to individual images, and all annotation data are stored in these JSON files.

In this tutorial we will focus on segmenting powder particles. AMPIS also includes an example for detecting satellites. The process for training models for powder particles and satellites is identical.


In [ ]:
EXPERIMENT_NAME = 'particle' # can be 'particle' or 'satellite'
root = Path('AMPIS','examples','powder') # path to folder  containing labels
json_path_train = Path(root,'data','via_2.0.8/', f'via_powder_{EXPERIMENT_NAME}_masks_training.json')  # path to training data
json_path_val = Path(root,'data','via_2.0.8/', f'via_powder_{EXPERIMENT_NAME}_masks_validation.json')  # path to validation data

assert json_path_train.is_file(), 'training file not found!'
assert json_path_val.is_file(), 'validation file not found!'

### Registration
Detectron2 requires that datasets be registered for later use.
Registration stores the name of the dataset and a function that can be used to retrieve the image paths and labels in a format that the model can use.

To make registration easier, AMPIS provides the get_ddicts() function, which can be used to load data from a VIA annotation file into a format that detectron2 can work with.

In [ ]:
DatasetCatalog.clear()  # resets catalog, helps prevent errors from running cells multiple times

# store names of datasets that will be registered for easier access later
dataset_train = f'{EXPERIMENT_NAME}_Train'
dataset_valid = f'{EXPERIMENT_NAME}_Val'

# register the training dataset
DatasetCatalog.register(dataset_train, 
                        lambda f = json_path_train: data_utils.get_ddicts(label_fmt='via2',  # annotations generated from vgg image annotator
                                                                          im_root=f,  # path to the training data json file
                                                                          dataset_class='Train'))  # indicates this is training data

# register the validation dataset. Same exact process as above
DatasetCatalog.register(dataset_valid, 
                        lambda f = json_path_val: data_utils.get_ddicts(label_fmt='via2',  # annotations generated from vgg image annotator
                                                                        im_root=f,  # path to validation data json file
                                                                        dataset_class='Validation'))  # indicates this is validation data
print(f'Registered Datasets: {list(DatasetCatalog.data.keys())}')

## There is also a metadata catalog, which stores the class names.
for d in [dataset_train, dataset_valid]:
    MetadataCatalog.get(d).set(**{'thing_classes': [EXPERIMENT_NAME]})

### Visualize labeled data
This allows for verification that the labels loaded correctly and make sense


Also, this is a great chance to admire my hand-drawn labels, which took a really really long time to make!

#### Training images
Since there are more images (especially for satellites) we will only view a subset

In [ ]:
np.random.seed(42960)
for i in np.random.choice(DatasetCatalog.get(dataset_train), 3, replace=False):
    visualize.display_ddicts(i, None, dataset_train, suppress_labels=True)

#### Validation images

In [ ]:
for i in DatasetCatalog.get(dataset_valid):
    visualize.display_ddicts(i, None, dataset_valid, suppress_labels=True)

## Model Configuration
This is where we specify the directory where the outputs are saved, various hyperparameters for the model, and more.

The complete set of configurations is listed in the [detectron2 documentation](https://detectron2.readthedocs.io/en/latest/modules/config.html#config-references), but some of the more relevant ones are specified here.

In [ ]:
cfg = get_cfg() # initialize cfg object
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))  # load default parameters for Mask R-CNN
cfg.INPUT.MASK_FORMAT = 'polygon'  # masks generated in VGG image annotator are polygons
cfg.DATASETS.TRAIN = (dataset_train,)  # dataset used for training model
cfg.DATASETS.TEST = (dataset_train, dataset_valid)  # we will look at the predictions on both sets after training
cfg.SOLVER.IMS_PER_BATCH = 1 # number of images per batch (across all machines)
cfg.SOLVER.CHECKPOINT_PERIOD = 400  # number of iterations after which to save model checkpoints
cfg.MODEL.DEVICE='cuda'  # 'cpu' to force model to run on cpu, 'cuda' if you have a compatible gpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Since we are training separate models for particles and satellites there is only one class output
cfg.TEST.DETECTIONS_PER_IMAGE = 400 if EXPERIMENT_NAME == 'particle' else 150  # maximum number of instances that can be detected in an image (this is fixed in mask r-cnn)
cfg.SOLVER.MAX_ITER = 2000  # maximum number of iterations to run during training
  # Increasing this may improve the training results, but will take longer to run (especially without a gpu!)

# model weights will be downloaded if they are not present
weights_path = Path('AMPIS','models','model_final_f10217.pkl')
if weights_path.is_file():
    print('Using locally stored weights: {}'.format(weights_path))
else:
    weights_path = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    print('Weights not found, weights will be downloaded from source: {}'.format(weights_path))
cfg.MODEL.WEIGHTs = str(weights_path)
cfg.OUTPUT_DIR = str(Path(f'{EXPERIMENT_NAME}_output'))
# make the output directory
os.makedirs(Path(cfg.OUTPUT_DIR), exist_ok=True)

## Training

After all that setup, we're finally ready to train the model!
With all of the parameters already specified in the config specified above, running training is very easy

In [ ]:
# note this cell generates a huge wall of text
trainer = DefaultTrainer(cfg)  # create trainer object from cfg
trainer.resume_or_load(resume=False)  # start training from iteration 0
trainer.train()  # train the model!

## Visualizing model predictions

In [ ]:
# load the weights of the model we want to use 
model_checkpoints = sorted(Path(cfg.OUTPUT_DIR).glob('*.pth'))  # paths to weights saved druing training
cfg.DATASETS.TEST = (dataset_train, dataset_valid)  # predictor requires this field to not be empty
cfg.MODEL.WEIGHTS = str(model_checkpoints[-1])  # use the last model checkpoint saved during training. If you want to see the performance of other checkpoints you can select a different index from model_checkpoints.
predictor = DefaultPredictor(cfg)  # create predictor object

### Single image
We can run the model on any image.
Note the image does not have to already be in a registered dataset.


In [ ]:
img_path = Path(root, 'data','images_png','Sc1Tile_001-005-000_0-000.png')
img = cv2.imread(str(img_path))
outs = predictor(img)
data_utils.format_outputs(img_path, dataset='test', pred=outs)
visualize.display_ddicts(ddict=outs,  # predictions to display
                                 outpath=None, dataset='Test',  # don't save figure
                                 gt=False,  # specifies format as model predictions
                                img_path=img_path)  # path to image


### All images in training and validation sets
We will save the results for later use.

In [ ]:
results = []
for ds in cfg.DATASETS.TEST:
    print(f'Dataset: {ds}')
    for dd in DatasetCatalog.get(ds):
        print(f'\tFile: {dd["file_name"]}')
        img = cv2.imread(dd['file_name'])  # load image
        outs = predictor(img)  # run inference on image
        
        # format results for visualization and store for later
        # note the use of format_outputs(), which ensures that the data is stored correctly for later
        results.append(data_utils.format_outputs(dd['file_name'], ds, outs))

        # visualize results
        visualize.display_ddicts(outs, None, ds, gt=False, img_path=dd['file_name'])

# save to disk
prediction_save_path = Path(f'{EXPERIMENT_NAME}-results.pickle')
with open(prediction_save_path, 'wb') as f:
    pickle.dump(results, f)

### Downloading the results (optional)
Uncomment and run the following cell to download the predictions and model weights to your own computer.

In [ ]:
# from google.colab.files import download
# download(model_checkpoints[-1]) # model weights (large file, may take awhile)
# download(prediction_save_path) # predictions

# Evaluating the results
This section can be run independently of model training.

Note you must still run the "Install detectron2 and AMPIS" section above to install the required libraries.

## Module imports

In [ ]:
# some of these are redundant, but allows this section to be run independently of the training section
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pickle
import seaborn as sns
import skimage.io

from ampis import analyze, data_utils
from ampis.applications import powder
from ampis.structures import InstanceSet
from ampis.visualize import display_iset

%matplotlib inline

## Loading data

For model evaluation we need both the ground truth and predicted labels.

If you ran the model to generate your own predicted labels, you may change the path to use your own predicted labels. However, by default we will use the predicted labels from the original study.

In [ ]:
## load ground truth labels
root = Path('AMPIS','examples','powder')
via_path = Path(root, 'data','via_2.0.8')

particles_gt_path_train = via_path / 'via_powder_particle_masks_training.json'
particles_gt_path_valid = via_path / 'via_powder_particle_masks_validation.json'

satellites_gt_path_train = via_path / 'via_powder_satellite_masks_training.json'
satellites_gt_path_valid = via_path / 'via_powder_satellite_masks_validation.json'

for path in [particles_gt_path_train, particles_gt_path_valid, satellites_gt_path_train, satellites_gt_path_valid]:
    assert path.is_file(), f'File not found : {path}'

# note that get_ddicts() loads the data in the standard detectron2 format
particles_gt_dd_train = data_utils.get_ddicts('via2', particles_gt_path_train, dataset_class='train')
particles_gt_dd_valid = data_utils.get_ddicts('via2', particles_gt_path_valid, dataset_class='validation')

satellites_gt_dd_train = data_utils.get_ddicts('via2', satellites_gt_path_train, dataset_class='train')
satellites_gt_dd_valid = data_utils.get_ddicts('via2', satellites_gt_path_valid, dataset_class='validation')

In [ ]:
# load predicted labels
particles_path = Path(root, 'data','sample_particle_outputs.pickle')
assert particles_path.is_file()

satellites_path = Path(root, 'data','sample_satellite_outputs.pickle')
assert satellites_path.is_file()

with open(particles_path, 'rb') as f:
    particle_pred = pickle.load(f)

with open(satellites_path, 'rb') as f:
    satellites_pred = pickle.load(f)

### Converting from "data-dicts" to InstanceSet objects

AMPIS uses the "InstanceSet" class for visualizing and evaluating the results, as well as for the final sample characterization. The InstanceSet objects provides functionality that is much more convenient to work with compared to the data-dictionary format used in detectron2.

The following cell converts the ground-truth and predicted annotations for each image to a list of InstanceSet objects. 

For the ground truth data we start with the training data, and then include the validation data at the end of the list. This is more compact than loading the training and validation data into separate lists.

In [ ]:
# Ground truth instance sets

iset_particles_gt = [InstanceSet().read_from_ddict(x,   # data 
                                                   inplace=False  # returns the set so it can be added to the list
                                                  ) for x in particles_gt_dd_train]

# instead of creating a separate list, we add the validation results to the training ones to make it easier later
iset_particles_gt.extend([InstanceSet().read_from_ddict(x, inplace=False) for x in particles_gt_dd_valid])

iset_satellites_gt = [InstanceSet().read_from_ddict(x, inplace=False) for x in satellites_gt_dd_train]
iset_satellites_gt.extend([InstanceSet().read_from_ddict(x, inplace=False) for x in satellites_gt_dd_valid])

# Predicted instance sets
iset_particles_pred = [InstanceSet().read_from_model_out(x, inplace=False) for x in particle_pred]
iset_satellites_pred = [InstanceSet().read_from_model_out(x, inplace=False) for x in satellites_pred]

### Matching the order of ground truth and predicted data
The ordering of the loaded data might be inconsistent. The analysis is easier when iset_satellites_gt and iset_particles_pred 
We want to rearrange the order of the instance sets in the ground truth and predicted lists so that the files are in the same order. This will make it easier later. In the following cell we match the ordering of the predicted instances to match that of the ground truth instances.

In [ ]:
iset_particles_gt, iset_particles_pred = analyze.align_instance_sets(iset_particles_gt, iset_particles_pred)
iset_satellites_gt, iset_satellites_pred = analyze.align_instance_sets(iset_satellites_gt, iset_satellites_pred)

# to verify that the filenames match we can print them out
for i, (gt, pred) in enumerate(zip(iset_particles_gt, iset_particles_pred)):
    pred.HFW = gt.HFW # predicted instances don't have specified HFW yet
    pred.HFW_units = gt.HFW_units
    assert Path(gt.filepath).name == Path(pred.filepath).name
    print(f'index {i}:  gt filename: {Path(gt.filepath).name}\t pred filename: {Path(pred.filepath).name}')
    pred.filepath = gt.filepath # the original AMPIS example had a different file organization than the colab
    # version. Because VIA uses relative paths it is not affected by this.
    # To prevent 'file not found' errors, update the file path for the images in the predicted instances
    # to match the true location of the file in Colab.


## Visualizing predicted labels on validation image

In [ ]:
# for now, the image has to be loaded separately
# updating the function to automatically find the image is on the to-do list
# it also appears the image must be loaded as an RGB image. This might have to do with the package versions on colab...
img = skimage.io.imread(iset_particles_pred[-1].filepath)
img = skimage.color.gray2rgb(img)
display_iset(img, iset_particles_pred[-1])

## Compute performance metrics

The standard metrics for evaluation provided in AMPIS are the detection and segmentation precision and recall. These results are described in detail in Section 2.3 of [the original paper](https://arxiv.org/abs/2101.01585). 

Basically, the ground truth and predicted instances are overlaid to determine which predicted instances correctly correspond to a ground truth instance by a criteria of minimum overlap. Detection precision and recall indicate how many instances were correctly matched. Segmentation precision and recall measure how *well* the predicted and ground-truth masks agree.

These metrics are computed and returned as a dictionary in the function det_seg_scores()

To keep things simple we will only look at powder particles for now. The process for evaluating the satellite masks is exactly the same.

In [ ]:
dss_particles = [analyze.det_seg_scores(gt, pred, size=gt.instances.image_size) 
                 for gt, pred in zip(iset_particles_gt, iset_particles_pred)]


### Visualize detection performance


In the visualiaztion, true positive masks (gt matches pred) are shown in purple, false positive masks (unmatched predicted masks) are shown in blue, and false negatives (unmatched ground truth instances) are shown in red.

In [ ]:
gt = iset_particles_gt[-3]
pred = iset_particles_pred[-3]
iset = gt
iset_det, colormap = analyze.det_perf_iset(gt, pred) # the results are returned as an InstanceSet for easy visualization
img = skimage.color.gray2rgb(skimage.io.imread(iset.filepath)) # again, in colab we have to convert to rgb for some reason
display_iset(img, iset=iset_det)

We can also plot the quantitative values for the detection precision and recall.

In [ ]:
labels = []
counts = {'train': 0, 'validation': 0}

# the filenames are not helpful, so we will map them to labels ie ('Train 1', 'Train 2', 'Validation 1', etc)
for iset in iset_particles_gt:
    counts[iset.dataset_class] += 1
    labels.append('{} {}'.format(iset.dataset_class, counts[iset.dataset_class]))

# x values are arbitrary, we just want 2 values, 1 for precision, 2 for recall
x=[*([1] * len(labels)), *([2] * len(labels))]
# y values are the bar heights
scores = [*[x['det_precision'] for x in dss_particles],
     *[x['det_recall'] for x in dss_particles]]

# since we are plotting precision and recall on the same plot we need 2 sets of labels
if len(labels) < len(x): # prevent length from changing if cell is re-run
  labels = labels * 2
print('x: ', x)
print('y: ', [np.round(x, decimals=2) for x in scores])
print('labels: ', labels)

fig, ax = plt.subplots(figsize=(6,3), dpi=150)
sns.barplot(x=x, y=scores, hue=labels, ax=ax)

ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel('detection score')
ax.set_xticklabels(['precision','recall'])

### Visualize segmentation performance
Similar to above, true positives, false positives, and false negatives are shown in purple, blue, and red. However, for the segmentation scores, true positives are pixels included in both the predicted segmentation match and the corresponding ground truth mask it matches to. False positives are pixels included in the predicted mask but not the ground truth mask. False negatives are pixels included in the ground truth masks that were missed by the model and are not included in the predicted masks.



In [ ]:
iset_seg, (colors, color_labels) = analyze.seg_perf_iset(gt, pred,)
display_iset(img, iset=iset_seg, apply_correction=True)

Unlike detection precision and recall, where each mask contributes 1 value for true/false positives or false negatives, the segmentation precision and recall contain true/false positives or false negatives for every individual pixel in the mask. Thus, instead of getting a single value for an image, we get a distribution of values, which can be displayed with the boxplot.

In [ ]:
import pandas as pd # this makes setting up the box plots easier
# y values are the bar heights
scores = [*[x['seg_precision'] for x in dss_particles],
     *[x['seg_recall'] for x in dss_particles]]

# due to the way seaborn handles data it is easier to move the data into a dataframe before generating the boxplot
dfs = []
for score, label, xi in zip(scores, labels, x):
  df_sub = pd.DataFrame({'score': score})
  df_sub['label'] = label
  df_sub['x'] = xi
  dfs.append(df_sub)
df = pd.concat(dfs)



fig, ax = plt.subplots(figsize=(6,3), dpi=150)
sns.boxplot(x='x', y='score', hue='label', data=df, ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel('segmentation score')
ax.set_xticklabels(['precision','recall'])

# Sample characterization
Now for the fun part! We can actually generate real physical measurements of samples from the segmentation masks generated before!

## Size distribution
Once we have the masks it is pretty trivial to compute various properties. With binary masks we can use [skimage regionprops](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops), which provides many convenient measurements out of the box. If there are any additional measurements you need, you can also access the masks directly and define your own methods. 

In [ ]:
# this takes ~30 seconds to run in colab
for iset in [*iset_particles_gt, *iset_particles_pred]:
    if iset.rprops is None:  # avoid re-computing regionprops if cell has already been run
        iset.compute_rprops()  # since rprops requires the masks to be uncompressed, this takes a bit longer to run
iset_particles_pred[-1].rprops.head()

In [ ]:
psd_results_gt = powder.psd(iset_particles_gt, plot=False, return_results=True)
psd_results_pred = powder.psd(iset_particles_pred, plot=False, return_results=True)

# make sure we are plotting the same thing...
assert psd_results_gt['x_label'] == psd_results_pred['x_label']
assert psd_results_gt['y_label'] == psd_results_pred['y_label']

In [ ]:
fig, ax = plt.subplots()
ax.plot(psd_results_gt['x'], psd_results_gt['y'], '--k', label='ground truth')
ax.plot(psd_results_pred['x'], psd_results_pred['y'], '-.m', label='predicted')
leg = ax.legend()
ax.set_xlabel(psd_results_gt['x_label'])
ax.set_ylabel(psd_results_gt['y_label'])

#### NOTE
The sample dataset does not have many particles. Thus, the distributions don't look really smooth. To increase the number of particles, the results include both masks from the training and validation sets. Because it includes training data, the agreement between ground truth and predicted distributions appears to be really good, but note that this isn't a fair comparison! This is just for demonstration purposes.

## Satellite Measurements
The claim to fame of the original study! This is first way of generating consistent, reproducible measurements for the fraction of satellited particles in powder samples.

The process here is fairly straightforward. We have masks for powder particles and masks for satellites. To match the satellites to their corresponding particles, we simply overlay the masks and look for intersections. Then, it is trivial to count the number of particles containing satellites. 

We have more labeled satellite images than particle images. We only want to keep images that have labels for both particles and satellites.
To help with the implementation, we can combine the masks for particles and satellites in the PowderSatelliteImage class

In [ ]:
# as well as aligning the order of files, the align_instance_sets function can also be used to remove extra 
# files that are present in  one dataset (in this case, labeled satellite images) but not the other (particles)

# ensure file ordering is the same, remove excess files for satellite labels
iset_particles_gt_ss, iset_satellites_gt_ss = analyze.align_instance_sets(iset_particles_gt, iset_satellites_gt)
iset_particles_pred_ss, iset_satellites_pred_ss = analyze.align_instance_sets(iset_particles_pred, iset_satellites_pred)

# The PowderSatelliteImage class contains InstanceSet objects for both masks and satellites for the same image 
psi_gt = []
psi_pred = []
for pg, pp, sg, sp in zip(iset_particles_gt_ss, iset_particles_pred_ss, iset_satellites_gt_ss, iset_satellites_pred_ss):
    files = [Path(x).name for x in [pg.filepath, pp.filepath, sg.filepath, sp.filepath]]
    assert all([x == files[0] for x in files])  # the files are in the same order and there are no excess files
    psi_gt.append(powder.PowderSatelliteImage(particles=pg, satellites=sg))
    psi_pred.append(powder.PowderSatelliteImage(particles=pp, satellites=sp))


In [ ]:
# compute_matches() finds which satellites belong to which particles in a given image
for gt, pred in zip(psi_gt, psi_pred):
    for psi in [gt, pred]:
        psi.compute_matches()

### Visualizing satellited particles

In [ ]:
gt = psi_gt[0]
pred = psi_pred[0]

np.random.seed(887890)
gt_idx = np.random.choice(list(gt.matches['match_pairs'].keys()), 3)
pred_idx = np.random.choice(list(pred.matches['match_pairs'].keys()), 3)

fig, ax = plt.subplots(2,3)


for i, (g, p) in enumerate(zip(gt_idx, pred_idx)):
    gt.visualize_particle_with_satellites(g, ax[0, i])
    pred.visualize_particle_with_satellites(p, ax[1, i])
ax[0,0].set_title('ground truth')
ax[1,0].set_title('predicted')

### Quantitative analysis of satellites
Again, note that the predicted results include predictions on the training images to increase the sample size. For real experiments, make sure you only use images that the model has not been trained on!

The summary is printed by the function.

In [ ]:
print('results')
results_gt = powder.satellite_measurements(psi_gt, print_summary=True, output_dict=True)
print('\n\n')
print('predicted results')
results_pred = powder.satellite_measurements(psi_pred, True, True)

The results are also returned as a dictionary so that they can be processed programatically.

In [ ]:
results_pred